In [20]:
import pandas as pd

In [21]:
data = pd.read_excel('../dataset/Quantathon_Data_2025.xlsx')

In [22]:
data = data.drop(index=data.index[:6])

In [23]:
# First dataset: columns A, B, and C
first_dataset = data.iloc[:, :3]

# Second dataset: columns E, F, and G
second_dataset = data.iloc[:, 4:7]


In [24]:
first_dataset.columns = ['Date', 'S&P 500', 'Bond Rate']
first_dataset

,Date,S&P 500,Bond Rate
6,2024-12-31 00:00:00,5881.63,4.208
7,2024-12-30 00:00:00,5906.94,4.182
8,2024-12-27 00:00:00,5970.84,4.178
9,2024-12-26 00:00:00,6037.59,4.215
10,2024-12-24 00:00:00,6040.04,4.2
...,...,...,...
4528,2007-01-12 00:00:00,1430.73,4.95
4529,2007-01-11 00:00:00,1423.82,4.945
4530,2007-01-10 00:00:00,1414.85,4.91
4531,2007-01-09 00:00:00,1412.11,4.91


In [25]:
second_dataset.columns = ['Date', 'PrDec', 'PrInc']
second_dataset

,Date,PrDec,PrInc
6,2024-12-31 00:00:00,0.082,0.10128
7,2024-12-24 00:00:00,0.0778,0.09324
8,2024-12-18 00:00:00,0.08909,0.14135
9,2024-12-11 00:00:00,0.07816,0.10151
10,2024-12-04 00:00:00,0.07441,0.09571
...,...,...,...
4528,NaN,NaN,NaN
4529,NaN,NaN,NaN
4530,NaN,NaN,NaN
4531,NaN,NaN,NaN


In [26]:
merged_dataset = pd.merge(first_dataset, second_dataset, on='Date', how='outer').dropna(how='all').sort_values(by='Date')

In [27]:
merged_dataset['PrDec'] = merged_dataset['PrDec'].infer_objects(copy=False).ffill().bfill()
merged_dataset['PrInc'] = merged_dataset['PrInc'].infer_objects(copy=False).ffill().bfill()
merged_dataset.to_csv('../dataset/Quantathon_Data_2025.csv', index=False)

In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

# Filter the data up to 2018
train_data = merged_dataset[merged_dataset['Date'] <= '2018-12-31']
test_data = merged_dataset[merged_dataset['Date'] > '2018-12-31']

# Prepare the training and test data
X_train = train_data[['Bond Rate', 'PrDec', 'PrInc']].values
y_train = train_data['S&P 500'].values
X_test = test_data[['Bond Rate', 'PrDec', 'PrInc']].values
y_test = test_data['S&P 500'].values

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)
y_train = imputer.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test = imputer.transform(y_test.reshape(-1, 1)).ravel()

# Define the model
model = LinearRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

# Determine buy or stop buying based on predictions
actions = []
for i in range(1, len(predictions)):
    if predictions[i] > predictions[i - 1]:
        actions.append('Buy')
    else:
        actions.append('Stop Buying')

# Add the first action as 'Hold' since we don't have a previous prediction to compare
actions.insert(0, 'Hold')

# Add the actions to the test_data dataframe
test_data['Action'] = actions

print(test_data[['Date', 'S&P 500', 'Action']])

Mean Squared Error: 6078269.915732836
R^2 Score: -7.147209922059302
           Date  S&P 500       Action
6809 2019-01-02  2510.03         Hold
6810 2019-01-03  2447.89  Stop Buying
6811 2019-01-04  2531.94          Buy
6812 2019-01-07  2549.69  Stop Buying
6813 2019-01-08  2574.41          Buy
...         ...      ...          ...
8314 2024-12-24  6040.04          Buy
8315 2024-12-26  6037.59          Buy
8316 2024-12-27  5970.84  Stop Buying
8317 2024-12-30  5906.94          Buy
8318 2024-12-31  5881.63  Stop Buying

[1510 rows x 3 columns]


C:\Users\Farhan\AppData\Local\Temp\ipykernel_28940\720563427.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Action'] = actions
